In [287]:
rv = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
rc = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

dataset = 'gowalla'     # Falta 5
dataset = 'foursquare' # Falta 1
dataset = 'brightkite' # Falta 146

In [288]:
def getAll(df, col=' Recall'):
    import itertools

    #settings = dict(map(lambda r: (str(r[0])+':'+str(r[1]), '-'), list(itertools.product(rv, rc))))
    settings = dict(map(lambda r: (str(r[0])+':'+str(r[1]), None), list(itertools.product(rc, rv))))
    settings_aux = dict(map(lambda row: (str(row[1]['Setting rv'])+':'+str(row[1][' Setting rc']), row[1][col]), df.iterrows()))
    settings.update(settings_aux)
    return settings

def findBest(df, col=' Recall'):
    best = df[col].max()
    #return df[df[col] == best]
    best = df[df[col] == best].tail(1)
    return best['Setting rv'].values[0], best[' Setting rc'].values[0], best[col].values[0]

def findConfig(df, rv, rc, col=' Recall'):
    best = df[col].max()
    #return df[df[col] == best]
    best = df[(df['Setting rv'] == rv) & (df[' Setting rc'] == rc)].tail(1)
    return rc, rv, best[col].values[0] if len(best) > 0 else 0
    
def mean(df, col=' Recall'):
    return '-', '-', df[col].mean()

def readFiles(path, col=' Recall', option=1, rv=0.05, rc=0.05):
    import os
    import pandas as pd
    from glob2 import glob
    XUSU = 0
    
    ls = []    
    for d in glob(path+"/*/", recursive = True):
        #d is 'Foursquare-nyc/output - validation - foursquare_user_6/'
        #print(d)
        f = d.split(' - ')[2][:-1]
        c = f.split('_')[2]
        f = d + f + '[Validation].csv'
        
        if not os.path.exists(f):
            #print("NAO ENCONTRADO:", f)
            XUSU += 1
            continue
        
        df = pd.read_csv(f, skiprows=[0])
        
        if option == 1:
            # Option 1
            ls.append( [c] + list(findBest(df, col)) )
        elif option == 2: 
            # Option 2
            data = getAll(df, col)
            data.update({'class': c})
            ls.append( data )
        elif option == 4:
            # Option 1
            ls.append( [c] + list(mean(df, col)) )
        else:  
            # Option 3
            ls.append( [c] + list(findConfig(df, rv, rc, col)) )
        
    print("NAO ENCONTRADOS:", XUSU)
    if option == 1 or option == 3 or option == 4:
        # Option 1, 3
        return pd.DataFrame(ls, columns=['class', 'rv', 'rc', col.strip()])
    elif option == 2:
        # Option 2
        return pd.DataFrame(ls)

In [289]:
# Option 1
result = readFiles(dataset)
print('------------------------------------------------------------------')
print('Melhores Recall (Média,Avg):', result['Recall'].mean(), "Median:", result['Recall'].median())
print('Contagem das Configurações:')
#result[['rv', 'rc']].value_counts() # rc:0.05 e rv:0.25 são as melhores!
result['Recall'].describe()

NAO ENCONTRADOS: 146
------------------------------------------------------------------
Melhores Recall (Média,Avg): 0.986158940397351 Median: 1.0
Contagem das Configurações:


count    151.000000
mean       0.986159
std        0.037396
min        0.740000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Recall, dtype: float64

In [251]:
import pandas as pd
with pd.option_context('display.max_rows', None):
    display(result)

,class,rv,rc,Recall


In [283]:
# Option 2
import pandas as pd
result = readFiles(dataset, option=2)
print('------------------------------------------------------------------')

# Médias por configuração:
print("Médias por configuração:")
mean = pd.DataFrame(result.mean())
display(mean.T)
print('------------------------------------------------------------------')
print('Best:', mean[:-1].max()[0])
print('Worse:', mean[:-1].min()[0])
#print("Média das médias:", pd.DataFrame(result.mean())[0][:-1].mean())
print('Missing:', result[result.columns[:-1]].T.isnull().sum().sum())
df = list()
for row in result[result.columns[:-1]].values:
    df = df + list(row)
print('Total:', len(df))
df = pd.DataFrame(df, columns=['Recall'])
print('Recall (Média,Avg):', df['Recall'].mean(), "Median:", df['Recall'].median())
display(df.describe())
print('------------------------------------------------------------------')
print('Todos os resultados:')
display(result)

NAO ENCONTRADOS: 5
------------------------------------------------------------------
Médias por configuração:


,0.01:0.01,0.01:0.05,0.01:0.1,0.01:0.15,0.01:0.2,0.01:0.25,0.05:0.01,0.05:0.05,0.05:0.1,0.05:0.15,...,0.2:0.15,0.2:0.2,0.2:0.25,0.25:0.01,0.25:0.05,0.25:0.1,0.25:0.15,0.25:0.2,0.25:0.25,class
0,0.920237,0.92661,0.930678,0.938915,0.942983,0.952475,0.927492,0.936678,0.932915,0.932678,...,0.487829,0.502471,0.504118,0.250102,0.359078,0.426291,0.480725,0.495041,0.50106,inf


------------------------------------------------------------------
Best: 0.9524745762711865
Worse: 0.25010169491525425
Missing: 331
Total: 10620
Recall (Média,Avg): 0.6696433083875983 Median: 0.83


,Recall
count,10289.000000
mean,0.669643
std,0.353557
min,0.000000
25%,0.350000
50%,0.830000
75%,1.000000
max,1.000000


------------------------------------------------------------------
Todos os resultados:


,0.01:0.01,0.01:0.05,0.01:0.1,0.01:0.15,0.01:0.2,0.01:0.25,0.05:0.01,0.05:0.05,0.05:0.1,0.05:0.15,...,0.2:0.15,0.2:0.2,0.2:0.25,0.25:0.01,0.25:0.05,0.25:0.1,0.25:0.15,0.25:0.2,0.25:0.25,class
0,0.88,0.94,0.94,0.81,1.00,1.00,1.00,0.94,0.94,0.94,...,0.56,0.31,0.31,0.12,0.06,0.06,0.50,0.25,0.25,5677
1,0.88,0.88,0.82,0.76,0.82,0.82,0.82,0.88,0.82,0.82,...,0.59,0.59,0.59,0.00,0.00,0.06,0.41,0.41,0.41,2660
2,0.77,0.92,0.92,0.92,0.92,0.92,1.00,0.92,0.92,0.92,...,0.69,0.69,0.69,0.08,0.62,0.69,0.69,0.69,0.69,107009
3,1.00,1.00,1.00,0.90,0.90,0.90,1.00,1.00,1.00,1.00,...,0.10,0.00,0.00,0.10,0.10,0.10,0.10,0.70,0.20,10153
4,1.00,1.00,1.00,0.90,0.90,1.00,1.00,0.90,1.00,0.90,...,0.10,0.10,0.20,0.00,0.00,0.00,0.00,0.00,0.00,26856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,0.86,0.79,0.79,0.86,0.86,0.86,1.00,1.00,1.00,1.00,...,0.50,0.14,NaN,0.00,0.00,0.00,0.64,0.14,NaN,142
291,0.89,0.84,0.84,0.84,1.00,1.00,0.95,0.95,0.89,0.89,...,0.00,NaN,NaN,0.84,0.58,NaN,NaN,NaN,NaN,5406
292,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,0.47,0.47,0.47,0.00,0.24,NaN,NaN,NaN,NaN,106154
293,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,0.70,0.70,0.70,0.10,0.55,0.05,0.50,0.50,0.50,28375


In [269]:
import pandas as pd
result = readFiles(dataset, option=2)
print('------------------------------------------------------------------')

# Médias por configuração:
print("Médias por configuração:")
result = pd.DataFrame(result.mean()).T.values[0][:-1]
#display(result)
import numpy as np
df = pd.DataFrame(np.reshape(result, (6, 6)), columns=rv, index=rc)
print(df.to_latex())

NAO ENCONTRADOS: 1
------------------------------------------------------------------
Médias por configuração:
\begin{tabular}{lrrrrrr}
\toprule
{} &      0.01 &      0.05 &      0.10 &      0.15 &      0.20 &      0.25 \\
\midrule
0.01 &  0.853958 &  0.888646 &  0.885573 &  0.877969 &  0.863750 &  0.845885 \\
0.05 &  0.842083 &  0.865052 &  0.867188 &  0.860833 &  0.844687 &  0.831354 \\
0.10 &  0.593646 &  0.637083 &  0.618646 &  0.624869 &  0.628377 &  0.618534 \\
0.15 &  0.433906 &  0.448579 &  0.457684 &  0.466968 &  0.496952 &  0.475847 \\
0.20 &  0.357760 &  0.426368 &  0.434545 &  0.440773 &  0.437572 &  0.430633 \\
0.25 &  0.306979 &  0.393915 &  0.399836 &  0.417616 &  0.416129 &  0.413768 \\
\bottomrule
\end{tabular}



In [140]:
new_row = list(result.columns[:-1].map(lambda x: x.split(':')[1])) + ['rc']
df1 = pd.DataFrame(new_row).T
df1.columns = list(result.columns)
df2 = pd.concat([df1,result.loc[:]], axis=0)
new_row = list(result.columns[:-1].map(lambda x: x.split(':')[0])) + ['rv']
df1 = pd.DataFrame(new_row).T
df1.columns = list(result.columns)
df2 = pd.concat([df1,df2.loc[:]], axis=0).reset_index(drop=True)
#print(df2.to_latex(index=False))
df2

,0.05:0.05,0.05:0.1,0.05:0.15,0.05:0.2,0.05:0.25,0.1:0.05,0.1:0.1,0.1:0.15,0.1:0.2,0.1:0.25,...,0.2:0.1,0.2:0.15,0.2:0.2,0.2:0.25,0.25:0.05,0.25:0.1,0.25:0.15,0.25:0.2,0.25:0.25,class
0,0.05,0.05,0.05,0.05,0.05,0.1,0.1,0.1,0.1,0.1,...,0.2,0.2,0.2,0.2,0.25,0.25,0.25,0.25,0.25,rv
1,0.05,0.1,0.15,0.2,0.25,0.05,0.1,0.15,0.2,0.25,...,0.1,0.15,0.2,0.25,0.05,0.1,0.15,0.2,0.25,rc
2,0.937474,0.93393,0.934491,0.928316,0.921965,0.736175,0.742035,0.74807,0.743298,0.731649,...,0.447878,0.490701,0.507352,0.513377,0.364735,0.426792,0.483016,0.502198,0.509275,inf


In [136]:
# Option 3
import itertools

settings = list(map(lambda r: (r[0],r[1]), list(itertools.product(rv, rc))))

for conf in settings:
    result = readFiles(dataset, option=3, rv=conf[0], rc=conf[1])
    print('------------------------------------------------------------------')
    print('rv', conf[0], '\trc', conf[1])
    print('Média - Recall:', result['Recall'].mean())

NAO ENCONTRADO: gowalla/output - validation - gowala_user_118449/gowala_user_118449[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_48534/gowala_user_48534[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_56524/gowala_user_56524[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_108841/gowala_user_108841[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_38089/gowala_user_38089[Validation].csv
------------------------------------------------------------------
rv 0.05 	rc 0.05
Média - Recall: 0.9374736842105266
NAO ENCONTRADO: gowalla/output - validation - gowala_user_118449/gowala_user_118449[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_48534/gowala_user_48534[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_56524/gowala_user_56524[Validation].csv
NAO ENCONTRADO: gowalla/output - validation - gowala_user_108841/gowala_user_108841[Validat

In [137]:
# Option 4
result = readFiles(dataset, option=4)
print('------------------------------------------------------------------')
print('Mean Average Recall (Média das médias):', result['Recall'].mean())

TypeError: 'DataFrame' object is not callable

---

In [138]:
#Ler por Classe:
c = 6
d = dataset+'/output - validation - '+dataset+'_user_'+str(c)+'/'
f = d.split(' - ')[2][:-1]
f = d + f + '[Validation].csv'
print(f)
df = pd.read_csv(f, skiprows=[0])
df

gowalla/output - validation - gowalla_user_6/gowalla_user_6[Validation].csv


FileNotFoundError: [Errno 2] No such file or directory: 'gowalla/output - validation - gowalla_user_6/gowalla_user_6[Validation].csv'

In [290]:
f = dataset + '_complete.csv'
df = pd.read_csv(f) #, skiprows=[0])

from automatize.preprocessing import countClasses_df
a = countClasses_df(df)
'Min trajs:', a.min(), 'Classes:', len(a)

Number of Samples: 7911
Samples by Class:

1770     89
6185     83
7522     80
2959     79
6906     77
         ..
29673    10
29598    10
9311     10
9129     10
11743    10
Length: 300, dtype: int64


('Min trajs:', 10, 'Classes:', 300)